# Got the medical documents from:
https://github.com/socd06/medical-nlp <br />
There's another tool someone made to get all medical terminologies on that link

In [1]:
def GetMedicalTerms(filePath): 
    
    lines = []
    with open(filePath) as f:
        for line in f:
            lines.append(line.strip().lower())
            
    return lines

# Initializing the LDA Model
https://radimrehurek.com/gensim/models/ldaseqmodel.html

In [2]:
def GetLDAModel(corpus, id2word,dictionary, numberOfTopics = 10, chunkSize=2000, passes=20, iterations=50):

    temp = dictionary[0]  

    return gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                   num_topics=numberOfTopics,
                                                   id2word=id2word,
                                                   chunksize=chunkSize,
                                                   iterations=iterations,
                                                   workers=5, # Num. Processing Cores - 1
                                                   passes=passes)

# Get the rule associations

In [3]:
from apyori import apriori
from nltk.stem import WordNetLemmatizer, PorterStemmer
from datetime import datetime
import pandas as pd  
from IPython.display import HTML 

def PrintRuleAssociation(lda, support, confidence, bow, lift, length = None):   
    porter = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    if len(bow) == 0:
        print('No topics found')
        return
    
    rules = apriori(bow,
                    min_support= support,
                    min_confidence= confidence,
                    min_lift= lift,
                    max_length= length)
    
    results = list(rules)
    
    df = pd.DataFrame(columns=('Left Hand Side',
                               'Right Hand Side',
                               'Support',
                               'Confidence',
                               'Lift',
                               'Count'))
    
    Support =[]
    Confidence = []
    Lift = []
    Items = []
    Antecedent = []
    Consequent=[]
    Count = []
      
    maxCount = 0
    for RelationRecord in results:
        for ordered_stat in RelationRecord.ordered_statistics:
            #print("count:", len(ordered_stat.items_base) + len(ordered_stat.items_add))
            #print("cons:", ordered_stat.items_add)
            if IsAntecedentDifferentFromConsequent(ordered_stat.items_base, ordered_stat.items_add):
                if maxCount < len(ordered_stat.items_base) + len(ordered_stat.items_add):
                    maxCount = len(ordered_stat.items_base) + len(ordered_stat.items_add)
                    
                consequences = list(ordered_stat.items_base)
                antecedent = list(ordered_stat.items_add)
                Support.append(RelationRecord.support)
                Antecedent.append(ordered_stat.items_base)
                Consequent.append(ordered_stat.items_add)
                Confidence.append(ordered_stat.confidence)
                Lift.append(ordered_stat.lift)
                Count.append(len(ordered_stat.items_base) + len(ordered_stat.items_add))
                                          
    df['Left Hand Side'] = list(map(set, Antecedent))
    df['Right Hand Side'] = list(map(set, Consequent))
    df['Support'] = Support
    df['Confidence'] = Confidence
    df['Lift'] = Lift
    df['Count'] = Count
    
    df.sort_values(by ='Count', ascending = False, inplace = True)
            
    #for index, row in df.iterrows():
    #    if row['Count'] >= maxCount-1:
    #        print(row)
    
    display(HTML(df.to_html()))
    #print(df)

In [4]:
def RemoveAllLowerCountAssociationRules():
    print("tbd")

In [5]:
from nltk.stem import LancasterStemmer

def SameStem(word1, word2, printStemmedWords = False):
    lancaster = LancasterStemmer()
    if printStemmedWords:
        print(lancaster.stem(word1), lancaster.stem(word2))

    return lancaster.stem(word1) == lancaster.stem(word2)

In [6]:
def IsAntecedentDifferentFromConsequent(Antecedent, Consequent):
    for ant in Antecedent:
        for cons in Consequent:
            if SameStem(ant, cons):
                return False
        
    return True
    

# Create the ngrams and set the ngram thresholds
min_count – Ignore all words and bigrams with total collected count lower than this value.
threshold – Represent a score threshold for forming the phrases. A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. Heavily depends on concrete scoring-function, see the scoring parameter.


In [7]:
import gensim
from gensim.models.phrases import Phrases,Phraser

def ngrams(words, minimumCount=5, threshold=15):
    bigram = Phrases(words,
                     min_count=minimumCount,
                     threshold=threshold)
    
    trigram = Phrases(bigram[words],
                      threshold=threshold)  

    bigram_mod = Phraser(bigram)
    trigram_mod = Phraser(trigram)
    return bigram_mod, trigram_mod


# Get the qualifying articles from the metadata
1. Qualifies if the publication date is within a given date range 
2. Has a pmc jcon file associated

In [8]:
from collections import defaultdict
import csv
from pathlib import Path
import json
import os.path

def GetData(startDate, endDate):
    data = defaultdict(list)   
    listofarticles = []
    try:
        with open('metadata.csv') as f_in:
            reader = csv.DictReader(f_in)
            for row in reader:
                if '-' not in row['publish_time']:
                    continue
                elif startDate > datetime.datetime.strptime(row['publish_time'], '%Y-%m-%d').date() or datetime.datetime.strptime(row['publish_time'], '%Y-%m-%d').date() > endDate:
                    continue
                if not row['pmc_json_files']:
                    continue   
            
                for json_path in row['pmc_json_files'].split(';'):
                    listofarticles.append(json_path)
                    data[row['cord_uid']] = True
                
    except ValueError:
        print("An error occurred: ", ValueError, " Please try again.")
    return listofarticles

# Get the textbodies from the list of pdfs
1. Remove all special characters

In [9]:
import json
import re

def GetTextBodies(listOfpdfs):    
    text = []    
    for json_path in listOfpdfs:
        if not os.path.exists(json_path.replace(" ", "")):
            continue
        
        with open(json_path.replace(" ", "")) as f_json:
            full_text_dict = json.load(f_json)
            textBody = []
            for paragraph_dict in full_text_dict['body_text']:          
                paragraph_text = re.sub(r'[^a-zA-Z_\s]+', '', paragraph_dict['text'])             
                textBody.append(paragraph_text)
                
        text.append(paragraph_text)
    return text

# Do the data pre-processing
1. remove stop words
2. lower case all words
3. Check to see if the word is within the list of medical terms given

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import WordNetLemmatizer, PorterStemmer

def CleanTheData(listOfDocs): 
    porter = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    stop_words = set(stopwords.words('english'))
  
    resultDocs = []
    words = set(nltk.corpus.words.words())
    medicalWords = GetMedicalTerms("vocab.txt")
    medicalStopWords = GetMedicalTerms("clinical-stopwords.txt")
    
    for doc in listOfDocs:
        result = []
        for word in doc.split(' '):  
            lowerCasedWord = word.lower()
            
            if lowerCasedWord in ['sarscov','sars-cov-2','coronavirus','sarscov']:
                lowerCasedWord = 'covid'
            
            if lowerCasedWord in "" or len(lowerCasedWord) <= 2 or lowerCasedWord in stop_words or lowerCasedWord not in medicalWords or lowerCasedWord in medicalStopWords:
                continue
                
            result.append(lowerCasedWord)
                
        resultDocs.append(result)
    return resultDocs

# Create the corpus
1. Create the dictionary with all words and word ids
2. Create the bi,tri, and quadgrams if applicable
3. remove extreme occurences of words

In [11]:
from gensim.corpora.dictionary import Dictionary
import spacy

def ConvertDataToCorpus(cleaned_data):

    dictionary = Dictionary(cleaned_data)
    bigramMod, trigramMod = ngrams(cleaned_data)
    
    ngram =  [trigramMod[bigramMod[review]] for review in cleaned_data]
       
    id2word = gensim.corpora.Dictionary(ngram)
    id2word.filter_extremes(no_below=10, no_above=0.90, keep_tokens=['covid'])
    
    id2word.compactify()
    
    corpus = [id2word.doc2bow(text) for text in ngram]
    
    return corpus, id2word, dictionary

# Get all qualifying topics generated from LDA model
1. Get all topics from the LDA model
2. Per each topic, check to see if the coherence score is greater than the lower bounce given
3. Per word, check to see if the word or a related word exists in the current topic, if it does, do not add
4. Per word, check to see if the probablity that the word is in the current topic is greater than the lower bound given
5. Do not consider topics with only one qualifying word

Notes: can read about coherence scores here: http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf
https://stackoverflow.com/questions/54762690/coherence-score-0-4-is-good-or-bad 
https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
https://levyomer.files.wordpress.com/2014/04/dependency-based-word-embeddings-acl-2014.pdf


In [24]:
import pprint as pp

def GetTopicTerms(lda,id2word, cleaned_data, lowerEndCoherenceScore, numberOfWords, minimumprobablity, windowSize, processes, printTopic = False):

    topTopics = lda.top_topics(corpus=None,
                               texts=cleaned_data,
                               dictionary=id2word,
                               coherence='c_v',
                               window_size=windowSize,
                               topn=numberOfWords, 
                               processes=processes)
    bow = []
    for topic in topTopics:
        transaction = []
        displayTransaction = []
        if printTopic:
            pp.pprint(topic)
        
        if topic[1] < lowerEndCoherenceScore:
            continue
            
        for word in topic[0]:
            l = []
            l.append(word[1])
            
            if word[0] > minimumprobablity and IsAntecedentDifferentFromConsequent(transaction, l):
                transaction.append(word[1])
                displayTransaction.append(word)
                
        if len(transaction) > 1:
            #pp.pprint(displayTransaction)
            bow.append(transaction)
            
    return bow

In [13]:
import datetime
def GetInformation():
    
    listOfpdfs= GetData(datetime.date(2020, 3, 1), datetime.date(2020, 3, 15))
    
    if len(listOfpdfs) == 0:
        print("No PDFs found under this topic")
        exit
        
    data = GetTextBodies(listOfpdfs)
    cleaned_data = CleanTheData(data)    
    corpus, id2word, dictionary = ConvertDataToCorpus(cleaned_data)
    
    lda = GetLDAModel(corpus,
                      id2word, 
                      dictionary,
                      numberOfTopics = 250,
                      chunkSize=2000,
                      passes=50, 
                      iterations=50)
    
    topics = GetTopicTerms(lda = lda, 
                           id2word = id2word,
                           cleaned_data=cleaned_data, 
                           lowerEndCoherenceScore=0, 
                           numberOfWords=30, 
                           minimumprobablity= 0.015,
                           windowSize= 15,
                           processes=10)
    
    PrintRuleAssociation(lda, 
                         support=0.01,
                         confidence=0.8,
                         bow=topics, 
                         lift = 1, 
                         length = None)



In [14]:
import datetime
from pprint import pprint

listOfpdfs= GetData(datetime.date(2020, 3, 1), datetime.date(2020, 4, 1))

if len(listOfpdfs) == 0:
    print("No PDFs found under this topic")
    exit
    
print('Amount of pdfs gathered: ', len(listOfpdfs))

Amount of pdfs gathered:  2273


In [15]:
data = GetTextBodies(listOfpdfs)

In [16]:
cleaned_data = CleanTheData(data)    

In [17]:
print(datetime.datetime.now())
corpus, id2word, dictionary = ConvertDataToCorpus(cleaned_data)

print(len(id2word))
print(datetime.datetime.now())

2021-01-04 14:07:40.139144
990
2021-01-04 14:07:41.789964


In [18]:
print(datetime.datetime.now())
lda = GetLDAModel(corpus,
                  id2word, 
                  dictionary,
                  numberOfTopics = 500,
                  chunkSize=2000,
                  passes=50, 
                  iterations=50)

print(datetime.datetime.now())

2021-01-04 14:07:41.796566
2021-01-04 14:09:12.911435


In [32]:
import pprint as pp
bow = GetTopicTerms(lda = lda, 
                    id2word = id2word,
                    cleaned_data=cleaned_data,
                    lowerEndCoherenceScore = 0,
                    numberOfWords=30, 
                    minimumprobablity= 0.02, 
                    windowSize= 110,
                    processes=10,
                    printTopic= False)
pp.pprint(bow)

/opt/anaconda3/lib/python3.8/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/opt/anaconda3/lib/python3.8/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


[['taken',
  'covid',
  'awareness',
  'matter',
  'manifestations',
  'significance',
  'potential',
  'patients'],
 ['approach', 'specificity', 'bacterial'],
 ['fast',
  'forward',
  'patients',
  'local',
  'following',
  'negative',
  'weeks',
  'years',
  'complexity'],
 ['activity',
  'covid',
  'spike',
  'target',
  'viral',
  'interaction',
  'decreased',
  'small',
  'therapeutic'],
 ['given',
  'board',
  'covid',
  'infected',
  'exposure',
  'identified',
  'investigation',
  'patients',
  'reported'],
 ['insight',
  'covid',
  'months',
  'fatal',
  'pulmonary',
  'note',
  'understanding',
  'responses',
  'therapeutic',
  'coming',
  'agents',
  'antibody'],
 ['fever',
  'patients',
  'physicians',
  'covid',
  'screening',
  'comparison',
  'evidence'],
 ['sampling', 'testing', 'community', 'covid', 'suspected', 'healthcare'],
 ['drugs',
  'covid',
  'effective',
  'patients',
  'viral',
  'safety',
  'therapy',
  'infection',
  'highly'],
 ['infectious_diseases', 'pro

In [33]:
len(bow)

280

In [34]:

import datetime
print(datetime.datetime.now())
PrintRuleAssociation(lda, support=0.005, confidence=0.8, bow=bow, lift = 35, length = None)
print(datetime.datetime.now())

2021-01-04 14:49:07.192501


,Left Hand Side,Right Hand Side,Support,Confidence,Lift,Count
117,"{covid, infection, transmission, virus}",{risk},0.007143,1.0,40.000000,5
109,"{covid, infection, patients, cancer}",{severe},0.007143,1.0,35.000000,5
101,"{covid, cancer}","{patients, infection, severe}",0.007143,1.0,140.000000,5
102,"{infection, cancer}","{patients, covid, severe}",0.007143,1.0,70.000000,5
103,"{infection, severe}","{patients, covid, cancer}",0.007143,1.0,140.000000,5
104,"{covid, infection, cancer}","{patients, severe}",0.007143,1.0,46.666667,5
106,"{patients, infection, cancer}","{covid, severe}",0.007143,1.0,70.000000,5
107,"{covid, infection, severe}","{patients, cancer}",0.007143,1.0,93.333333,5
108,"{patients, infection, severe}","{covid, cancer}",0.007143,1.0,140.000000,5
105,"{covid, patients, cancer}","{infection, severe}",0.007143,1.0,140.000000,5


2021-01-04 14:49:11.829156
